In [1]:
import pandas as pd
from sklearn.cluster import KMeans
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import  cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import sklearn.tree as tree
from copy import deepcopy
from analyze_tools import *

In [21]:
untoucheddf = pd.read_csv('../data/clean_game_data.csv', index_col=0)

In [2]:

df = pd.read_csv('../data/clean_game_data.csv', index_col=0).drop(columns=['Year', 'Game ID'])
df['Victory'] = df['Victory'].astype(int)
top_df = deepcopy(df[df['Role'] == 'TOP'].select_dtypes(include='number')).astype('float64').drop(columns=['Victory'])
mid_df = deepcopy(df[df['Role'] == 'MID'].select_dtypes(include='number')).astype('float64').drop(columns=['Victory'])
jg_df =  deepcopy(df[df['Role'] == 'JUNGLE'].select_dtypes(include='number')).astype('float64').drop(columns=['Victory'])
adc_df =  deepcopy(df[df['Role'] == 'ADC'].select_dtypes(include='number')).astype('float64').drop(columns=['Victory'])
sup_df = deepcopy(df[df['Role'] == 'SUPPORT'].select_dtypes(include='number')).astype('float64').drop(columns=['Victory'])

scaler = StandardScaler()
top_df[:] = scaler.fit_transform(top_df)
mid_df[:] = scaler.fit_transform(mid_df)
jg_df[:] = scaler.fit_transform(jg_df)
adc_df[:] = scaler.fit_transform(adc_df)
sup_df[:] = scaler.fit_transform(sup_df)

pca3_mid = PCA(n_components=3)
pca3_top = PCA(n_components=3)
pca3_jg = PCA(n_components=3)
pca3_adc = PCA(n_components=3)
pca3_sup = PCA(n_components=3)

pca3_mid_df = pd.DataFrame(pca3_mid.fit_transform(mid_df), columns=['PC1', 'PC2', 'PC3'])
pca3_top_df = pd.DataFrame(pca3_top.fit_transform(top_df), columns=['PC1', 'PC2', 'PC3'])
pca3_jg_df = pd.DataFrame(pca3_jg.fit_transform(jg_df), columns=['PC1', 'PC2', 'PC3'])
pca3_adc_df = pd.DataFrame(pca3_adc.fit_transform(adc_df), columns=['PC1', 'PC2', 'PC3'])
pca3_sup_df = pd.DataFrame(pca3_sup.fit_transform(sup_df), columns=['PC1', 'PC2', 'PC3'])



In [3]:
n_clusters = 4
top_kmeans = KMeans(n_clusters=n_clusters,random_state=42)
pred = top_kmeans.fit_predict(pca3_top_df)
new_top_df = deepcopy(top_df)
new_top_df['Cluster'] = pred
new_top_df['Victory'] = df[df['Role'] == 'TOP']['Victory']
print(new_top_df.groupby('Cluster').mean()['Victory'] * 100)

mid_kmeans = KMeans(n_clusters=n_clusters,random_state=42)
pred = mid_kmeans.fit_predict(pca3_mid_df)
new_mid_df = deepcopy(mid_df)
new_mid_df['Cluster'] = pred
new_mid_df['Victory'] = df[df['Role'] == 'MID']['Victory']
print(new_mid_df.groupby('Cluster').mean()['Victory'] * 100)

jg_kmeans = KMeans(n_clusters=n_clusters,random_state=42)
pred = jg_kmeans.fit_predict(pca3_jg_df)
new_jg_df = deepcopy(jg_df)
new_jg_df['Cluster'] = pred
new_jg_df['Victory'] = df[df['Role'] == 'JUNGLE']['Victory']
print(new_jg_df.groupby('Cluster').mean()['Victory'] * 100)

adc_kmeans = KMeans(n_clusters=n_clusters,random_state=42)
pred = adc_kmeans.fit_predict(pca3_adc_df)
new_adc_df = deepcopy(adc_df)
new_adc_df['Cluster'] = pred
new_adc_df['Victory'] = df[df['Role'] == 'ADC']['Victory']
print(new_adc_df.groupby('Cluster').mean()['Victory'] * 100)

sup_kmeans = KMeans(n_clusters=n_clusters,random_state=42)
pred = sup_kmeans.fit_predict(pca3_sup_df)
new_sup_df = deepcopy(sup_df)
new_sup_df['Cluster'] = pred
new_sup_df['Victory'] = df[df['Role'] == 'SUPPORT']['Victory']
print(new_sup_df.groupby('Cluster').mean()['Victory'] * 100)


Cluster
0    50.687876
1    80.716029
2    19.072502
3    60.731707
Name: Victory, dtype: float64
Cluster
0    67.553957
1    19.852741
2    50.101523
3    84.953233
Name: Victory, dtype: float64
Cluster
0    65.441696
1    15.160891
2    59.164292
3    87.632777
Name: Victory, dtype: float64
Cluster
0    16.138763
1    67.427123
2    43.287800
3    90.080972
Name: Victory, dtype: float64
Cluster
0    54.912281
1    83.774969
2    71.197411
3    23.309241
Name: Victory, dtype: float64


In [4]:
# Assigning the cluster to the original dataframe for each player for each role in each game

clusterdf = pd.read_csv('../data/clean_game_data.csv', index_col=0)
clusterdf['Cluster'] = -1
clusterdf.loc[df['Role'] == 'TOP', 'Cluster'] = top_kmeans.predict(pca3_top_df)
clusterdf.loc[df['Role'] == 'MID', 'Cluster'] = mid_kmeans.predict(pca3_mid_df)
clusterdf.loc[df['Role'] == 'JUNGLE', 'Cluster'] = jg_kmeans.predict(pca3_jg_df)
clusterdf.loc[df['Role'] == 'ADC', 'Cluster'] = adc_kmeans.predict(pca3_adc_df)
clusterdf.loc[df['Role'] == 'SUPPORT', 'Cluster'] = sup_kmeans.predict(pca3_sup_df)

In [5]:
clusterdf

,Side,Region,Year,Game ID,Team,Victory,Game Duration,Player,Role,Kills,...,Penta kills,GD@15,CSD@15,XPD@15,LVLD@15,Damage dealt to turrets,Total heal,Time ccing others,Total damage taken,Cluster
Column,,,,,,,,,,,,,,,,,,,,,
0,Blue Side,WORLDS,2023,53622,Weibo Gaming,False,30.517,TheShy,TOP,1,...,0,-243,-30,-64,0,968,8053,12,27242,2
1,Blue Side,WORLDS,2023,53622,Weibo Gaming,False,30.517,WeiWei,JUNGLE,3,...,0,212,-7,-11,0,0,14875,39,30384,0
2,Blue Side,WORLDS,2023,53622,Weibo Gaming,False,30.517,xiaohu,MID,1,...,0,363,9,511,1,1622,3561,8,14642,2
3,Blue Side,WORLDS,2023,53622,Weibo Gaming,False,30.517,Crisp,ADC,0,...,0,-1002,-45,-580,0,0,3640,7,17807,0
4,Blue Side,WORLDS,2023,53622,Weibo Gaming,False,30.517,Light,SUPPORT,0,...,0,435,15,352,1,379,7932,23,11032,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5,Red Side,LEC,2020,20718,MAD Lions,False,34.317,Orome,TOP,0,...,0,771,-5,954,1,1363,2639,30,24106,2
6,Red Side,LEC,2020,20718,MAD Lions,False,34.317,Shad0w,JUNGLE,4,...,0,163,-23,-26,0,826,11991,14,26162,0
7,Red Side,LEC,2020,20718,MAD Lions,False,34.317,Humanoid,MID,2,...,0,48,-2,-97,-1,102,6409,8,30352,1


Learning cluster interation, distribution for each team comp.

In [15]:

x = np.zeros((len(clusterdf['Game ID'].unique()), 10))
y = np.zeros(len(clusterdf['Game ID'].unique()))

np.random.seed(42)
for i, gameid in enumerate(clusterdf['Game ID'].unique()):
    game = clusterdf[clusterdf['Game ID'] == gameid]
    if len(game) != 10:
        raise ValueError('Game with id {} has {} players'.format(gameid, len(game)))
    
    top_blue = game[(game['Role'] == 'TOP') & (game['Side'] == 'Blue Side')]['Cluster'].values[0]
    mid_blue = game[(game['Role'] == 'MID') & (game['Side'] == 'Blue Side')]['Cluster'].values[0]
    jg_blue = game[(game['Role'] == 'JUNGLE') & (game['Side'] == 'Blue Side')]['Cluster'].values[0]
    adc_blue = game[(game['Role'] == 'ADC') & (game['Side'] == 'Blue Side')]['Cluster'].values[0]
    sup_blue = game[(game['Role'] == 'SUPPORT') & (game['Side'] == 'Blue Side')]['Cluster'].values[0]

    top_red = game[(game['Role'] == 'TOP') & (game['Side'] == 'Red Side')]['Cluster'].values[0]
    mid_red = game[(game['Role'] == 'MID') & (game['Side'] == 'Red Side')]['Cluster'].values[0]
    jg_red = game[(game['Role'] == 'JUNGLE') & (game['Side'] == 'Red Side')]['Cluster'].values[0]
    adc_red = game[(game['Role'] == 'ADC') & (game['Side'] == 'Red Side')]['Cluster'].values[0]
    sup_red = game[(game['Role'] == 'SUPPORT') & (game['Side'] == 'Red Side')]['Cluster'].values[0]

    # Formating into a vector of cluster attribution for each player in the game and the result of the game
    result = game[(game['Role'] == 'TOP') & (game['Side'] == 'Blue Side')]['Victory'].values[0].astype(int)
    # Randomizing the side attribution
    if np.random.rand() > 0.5:
        vector = np.array([top_red, mid_red, jg_red, adc_red, sup_red, top_blue, mid_blue, jg_blue, adc_blue, sup_blue])
        result = 1 - result
    else:
        vector = np.array([top_blue, mid_blue, jg_blue, adc_blue, sup_blue, top_red, mid_red, jg_red, adc_red, sup_red])
    x[i] = vector
    y[i] = result

    

In [16]:
# Learning a model to predict the result of the game based on the cluster attribution of the players

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
clf = RandomForestClassifier(random_state=42)
clf.fit(x_train, y_train)
print('Random Forest Classifier')
print('Train score:', clf.score(x_train, y_train))
print('Test score:', clf.score(x_test, y_test))
print('Cross validation score:', cross_val_score(clf, x, y).mean())


Random Forest Classifier
Train score: 0.9865577227200844
Test score: 0.9135932560590094
Cross validation score: 0.9042818778816699


In [30]:
# Test on 2024 data

df2024 = pd.read_csv('../data/game_data2024.csv', index_col=0)
# Harmonizing columns with the previous dataframe (has possibly more unknown columns)
df2024 = df2024[untoucheddf.columns]
df2024predict = deepcopy(df2024).drop(columns=['Year', 'Game ID'])

df2024['Cluster'] = -1

print(pca3_top.feature_names_in_)
print(df2024predict[df2024['Role'] == 'TOP'].select_dtypes(include='number').columns)
# verify that the columns are the same
print(np.all(pca3_top.feature_names_in_ == df2024predict[df2024['Role'] == 'TOP'].select_dtypes(include='number').columns))

df2024.loc[df2024['Role'] == 'TOP', 'Cluster'] = top_kmeans.predict(pca3_top.transform(df2024predict[df2024['Role'] == 'TOP'].select_dtypes(include='number')))
df2024.loc[df2024['Role'] == 'MID', 'Cluster'] = mid_kmeans.predict(pca3_mid.transform(df2024predict[df2024['Role'] == 'MID'].select_dtypes(include='number')))
df2024.loc[df2024['Role'] == 'JUNGLE', 'Cluster'] = jg_kmeans.predict(pca3_jg.transform(df2024predict[df2024['Role'] == 'JUNGLE'].select_dtypes(include='number')))
df2024.loc[df2024['Role'] == 'ADC', 'Cluster'] = adc_kmeans.predict(pca3_adc.transform(df2024predict[df2024['Role'] == 'ADC'].select_dtypes(include='number')))
df2024.loc[df2024['Role'] == 'SUPPORT', 'Cluster'] = sup_kmeans.predict(pca3_sup.transform(df2024predict[df2024['Role'] == 'SUPPORT'].select_dtypes(include='number')))

df2024


['Game Duration' 'Kills' 'Deaths' 'Assists' 'KDA' 'CS'
 "CS in Team's Jungle" 'CSM' 'Golds' 'GPM' 'GOLD%' 'Vision Score'
 'Wards placed' 'Wards destroyed' 'Control Wards Purchased' 'VSPM' 'WPM'
 'VWPM' 'WCPM' 'VS%' 'Total damage to Champion' 'Physical Damage'
 'Magic Damage' 'True Damage' 'DPM' 'DMG%' 'K+A Per Minute' 'KP%'
 'Solo kills' 'Double kills' 'Triple kills' 'Quadra kills' 'Penta kills'
 'GD@15' 'CSD@15' 'XPD@15' 'LVLD@15' 'Damage dealt to turrets'
 'Total heal' 'Time ccing others' 'Total damage taken']
Index(['Game Duration', 'Kills', 'Deaths', 'Assists', 'KDA', 'CS',
       'CS in Team's Jungle', 'CSM', 'Golds', 'GPM', 'GOLD%', 'Vision Score',
       'Wards placed', 'Wards destroyed', 'Control Wards Purchased', 'VSPM',
       'WPM', 'VWPM', 'WCPM', 'VS%', 'Total damage to Champion',
       'Physical Damage', 'Magic Damage', 'True Damage', 'DPM', 'DMG%',
       'K+A Per Minute', 'KP%', 'Solo kills', 'Double kills', 'Triple kills',
       'Quadra kills', 'Penta kills', 'GD@15'

c:\Users\Shirakagi\miniconda3\envs\dac\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but KMeans was fitted with feature names
  warnings.warn(
c:\Users\Shirakagi\miniconda3\envs\dac\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but KMeans was fitted with feature names
  warnings.warn(
c:\Users\Shirakagi\miniconda3\envs\dac\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but KMeans was fitted with feature names
  warnings.warn(
c:\Users\Shirakagi\miniconda3\envs\dac\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but KMeans was fitted with feature names
  warnings.warn(
c:\Users\Shirakagi\miniconda3\envs\dac\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but KMeans was fitted with feature names
  warnings.warn(


,Side,Region,Year,Game ID,Team,Victory,Game Duration,Player,Role,Kills,...,Penta kills,GD@15,CSD@15,XPD@15,LVLD@15,Damage dealt to turrets,Total heal,Time ccing others,Total damage taken,Cluster
0,Blue Side,LEC,2024,57862,G2 Esports,False,28.517,Brokenblade,TOP,1.0,...,0.0,-471.0,0.0,435.0,1.0,293.0,3226.0,43.0,33743.0,3
1,Blue Side,LEC,2024,57862,G2 Esports,False,28.517,Yike,JUNGLE,1.0,...,0.0,-387.0,-5.0,-281.0,0.0,2657.0,14693.0,15.0,30359.0,3
2,Blue Side,LEC,2024,57862,G2 Esports,False,28.517,Caps,MID,5.0,...,0.0,1125.0,-1.0,458.0,0.0,5220.0,3372.0,18.0,20231.0,0
3,Blue Side,LEC,2024,57862,G2 Esports,False,28.517,Hans sama,ADC,3.0,...,0.0,361.0,-1.0,204.0,0.0,5856.0,2847.0,3.0,16606.0,1
4,Blue Side,LEC,2024,57862,G2 Esports,False,28.517,Mikyx,SUPPORT,1.0,...,0.0,-528.0,4.0,-635.0,-1.0,2286.0,943.0,20.0,16227.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5,Red Side,LCS,2024,53709,100 Thieves,True,34.917,Sniper,TOP,2.0,...,0.0,312.0,18.0,-63.0,0.0,5756.0,5000.0,34.0,22879.0,3
6,Red Side,LCS,2024,53709,100 Thieves,True,34.917,River,JUNGLE,4.0,...,0.0,670.0,5.0,238.0,0.0,2016.0,18183.0,26.0,36494.0,3
7,Red Side,LCS,2024,53709,100 Thieves,True,34.917,Quid,MID,5.0,...,0.0,553.0,22.0,1264.0,1.0,6128.0,7223.0,19.0,22071.0,0
8,Red Side,LCS,2024,53709,100 Thieves,True,34.917,Meech,ADC,4.0,...,0.0,-977.0,-31.0,-1715.0,-2.0,3762.0,10294.0,32.0,14623.0,1


In [22]:




x2024 = np.zeros((len(df2024['Game ID'].unique()), 10))
y2024 = np.zeros(len(df2024['Game ID'].unique()))

for i, gameid in enumerate(df2024['Game ID'].unique()):
    game = df2024[df2024['Game ID'] == gameid]
    if len(game) != 10:
        raise ValueError('Game with id {} has {} players'.format(gameid, len(game)))
    
    top_blue = game[(game['Role'] == 'TOP') & (game['Side'] == 'Blue Side')]['Cluster'].values[0]
    mid_blue = game[(game['Role'] == 'MID') & (game['Side'] == 'Blue Side')]['Cluster'].values[0]
    jg_blue = game[(game['Role'] == 'JUNGLE') & (game['Side'] == 'Blue Side')]['Cluster'].values[0]
    adc_blue = game[(game['Role'] == 'ADC') & (game['Side'] == 'Blue Side')]['Cluster'].values[0]
    sup_blue = game[(game['Role'] == 'SUPPORT') & (game['Side'] == 'Blue Side')]['Cluster'].values[0]

    top_red = game[(game['Role'] == 'TOP') & (game['Side'] == 'Red Side')]['Cluster'].values[0]
    mid_red = game[(game['Role'] == 'MID') & (game['Side'] == 'Red Side')]['Cluster'].values[0]
    jg_red = game[(game['Role'] == 'JUNGLE') & (game['Side'] == 'Red Side')]['Cluster'].values[0]
    adc_red = game[(game['Role'] == 'ADC') & (game['Side'] == 'Red Side')]['Cluster'].values[0]
    sup_red = game[(game['Role'] == 'SUPPORT') & (game['Side'] == 'Red Side')]['Cluster'].values[0]

    result = game[(game['Role'] == 'TOP') & (game['Side'] == 'Blue Side')]['Victory'].values[0].astype(int)
    vector = np.array([top_blue, mid_blue, jg_blue, adc_blue, sup_blue, top_red, mid_red, jg_red, adc_red, sup_red])
    x2024[i] = vector
    y2024[i] = result


Index(['Side', 'Region', 'Year', 'Game ID', 'Team', 'Victory', 'Game Duration',
       'Player', 'Role', 'Kills', 'Deaths', 'Assists', 'KDA', 'CS',
       'CS in Team's Jungle', 'CSM', 'Golds', 'GPM', 'GOLD%', 'Vision Score',
       'Wards placed', 'Wards destroyed', 'Control Wards Purchased', 'VSPM',
       'WPM', 'VWPM', 'WCPM', 'VS%', 'Total damage to Champion',
       'Physical Damage', 'Magic Damage', 'True Damage', 'DPM', 'DMG%',
       'K+A Per Minute', 'KP%', 'Solo kills', 'Double kills', 'Triple kills',
       'Quadra kills', 'Penta kills', 'GD@15', 'CSD@15', 'XPD@15', 'LVLD@15',
       'Damage dealt to turrets', 'Total heal', 'Time ccing others',
       'Total damage taken'],
      dtype='object')


KeyError: 'Cluster'

In [18]:
clf = RandomForestClassifier(random_state=42)
clf.fit(x, y)

print('Random Forest Classifier')
print('Test score:', clf.score(x2024, y2024))

Random Forest Classifier


NameError: name 'x2024' is not defined